# Age-gender-race model with multisource data

## config

In [ ]:
IMAGE_SIZE = 56
BATCH_SIZE = 256
WEIGHT_INIT = 0.05
INIT_LR = 1e-3
DECAY_LR = 0.95
EPOCHS = 80
KEEP_PROB = 0.5
DECAY_STEP = 1000
WEIGHT_DECAY = 1e-4

MODEL_DIR = '/content/drive/MyDrive/3033proj/weights'
RACE_FOLDER = '/content/drive/MyDrive/3033proj/'
AGE_AND_GENDER_FOLDER = '/content/drive/MyDrive/3033proj/'


In [ ]:
import tensorflow as tf
session = tf.compat.v1.Session()

# Utils

In [ ]:
import tensorflow as tf
import numpy as np
import os

# import config


def batch_norm(x, n_out, phase_train=True, scope='bn'):
    """
    Batch normalization on convolutional maps.
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Variable, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.compat.v1.variable_scope(scope):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]), name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]), name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0, 1, 2], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed


def conv(name, x, filter_size, in_filters, out_filters, strides):
    with tf.compat.v1.variable_scope(name):
        size = [filter_size, filter_size, in_filters, out_filters]
        init = tf.compat.v1.truncated_normal_initializer(stddev=WEIGHT_INIT)
        filters = tf.compat.v1.get_variable('DW', size, tf.float32, init)

        return tf.nn.conv2d(x, filters, [1, strides, strides, 1], 'SAME')


def relu(x, leakiness=0.0):
    return tf.where(tf.less(x, 0.0), leakiness * x, x, name='leaky_relu')


def FC(name, x, output_dim, keep_rate, activation='relu'):
    assert (activation == 'relu') or (activation == 'softmax') or (activation == 'linear')
    with tf.compat.v1.variable_scope(name):
        dim = x.get_shape().as_list()

        # flatten
        dim = np.prod(dim[1:])
        x = tf.reshape(x, [-1, dim])

        # init bias, weight
        W = tf.compat.v1.get_variable('DW', [x.get_shape()[1], output_dim], initializer=tf.compat.v1.truncated_normal_initializer(
                                stddev=WEIGHT_INIT))

        b = tf.compat.v1.get_variable('bias', [output_dim], initializer=tf.constant_initializer())

        x = tf.compat.v1.nn.xw_plus_b(x, W, b)

        # Activation
        if activation == 'relu':
            x = relu(x)
        else:
            if activation == 'softmax':
                x = tf.nn.softmax(x)

        if activation != 'relu':
            return x
        else:
            return tf.nn.dropout(x, keep_rate)


def max_pool(x, filter_size, strides):
    return tf.nn.max_pool(x, [1, filter_size, filter_size, 1], [1, strides, strides, 1], 'SAME')


def VGG_ConvBlock(name, x, in_filters, out_filters, repeat, strides, phase_train):
    with tf.compat.v1.variable_scope(name):
        for layer in range(repeat):
            scope_name = name + '_' + str(layer)
            x = conv(scope_name, x, 3, in_filters, out_filters, strides)
            x = batch_norm(x, out_filters, phase_train)
            x = relu(x)

            in_filters = out_filters

        x = max_pool(x, 2, 2)
        return x


def get_one_hot_vector(num_classes, class_idx):
    """
        Return tensor of shape (num_classes, )
    """
    result = np.zeros(num_classes)
    result[class_idx] = 1.0

    return result

# Data 

In [ ]:
import numpy as np
import cv2
import argparse

# from config import SMILE_FOLDER, AGE_AND_GENDER_FOLDER
# import config as cf


def getRaceImage(trainable=True):
    print('==================================================================')
    print('\nLoading Race image datasets.....')
    X1 = np.load(RACE_FOLDER + 'data_3_utk.npy', allow_pickle=True)
    # X2 = np.load(SMILE_FOLDER + 'test.npy', allow_pickle=True)
    print('Done! ')

    train_data = []
    # test_data = []

    for i in range(X1.shape[0]):
        train_data.append(X1[i])

    # for i in range(X2.shape[0]):
    #     test_data.append(X2[i])

    if trainable:
        print('Number of race train data: ', str(len(train_data)))
    else:
        print('Number of smile test data: ', str(len(test_data)))

    return train_data


def getAgeImage(trainable=True):
    print('==================================================================')
    print('\nLoading age image datasets.....')
    X1 = np.load(AGE_AND_GENDER_FOLDER + 'data_3.npy', allow_pickle=True)
    # X2 = np.load(AGE_AND_GENDER_FOLDER + 'test_age.npy', allow_pickle=True)
    np.random.shuffle(X1)
    train_data = []
    # test_data = []

    for i in range(45000):
        train_data.append(X1[i])

    # for i in range(X2.shape[0]):
    #     test_data.append(X2[i])

    print('Done!')
    if trainable:
        print('Number of age train data: ', str(len(train_data)))
    else:
        print('Number of age test data: ', str(len(test_data)))

    return train_data #, test_data


def getGenderImage(trainable=True):
    print('==================================================================')
    print('\nLoading gender image datasets.....')
    X1 = np.load(AGE_AND_GENDER_FOLDER + 'data_3.npy', allow_pickle=True)
    # X2 = np.load(AGE_AND_GENDER_FOLDER + 'test_gender.npy', allow_pickle=True)
    np.random.shuffle(X1)
    train_data = []
    # test_data = []

    for i in range(45000):
        train_data.append(X1[i])

    # for i in range(X2.shape[0]):
    #     test_data.append(X2[i])

    print('Done!')
    if trainable:
        print('Number of gender train data: ', str(len(train_data)))
    else:
        print('Number of gender test data: ', str(len(test_data)))

    return train_data #, test_data


def draw_labels_and_boxes(img, boxes, labels, margin=0):
    for i in range(len(labels)):
        # get the bounding box coordinates
        left, top, right, bottom = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
        width = right - left
        height = bottom - top
        img_h, img_w = img.shape[:2]

        x1 = max(int(left - margin * width), 0)
        y1 = max(int(top - margin * height), 0)
        x2 = min(int(right + margin * width), img_w - 1)
        y2 = min(int(bottom + margin * height), img_h - 1)

        # Color red
        color = (0, 0, 255)

        # classify label according to result
        race_label, age_label, gender_label = labels[i]

        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        text = '{} {} {}'.format(gender_label, age_label, race_label)
        cv2.putText(img, text, (left - 35, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    return img


def get_arguments():
    arg = argparse.ArgumentParser()
    arg.add_argument('-i', '--image_path', help='path to the image')
    arg.add_argument('-v', '--video_path', help='path to the video file')
    arg.add_argument('-m', '--margin', help='margin around face', default=0.0)
    return arg.parse_args()


def crop_face(image, result):
    nb_detected_faces = len(result)

    cropped_face = np.empty((nb_detected_faces, cf.IMAGE_SIZE, cf.IMAGE_SIZE, 1))
    boxes = []
    # loop through detected face
    for i in range(nb_detected_faces):
        # coordinates of boxes
        bounding_box = result[i]['box']
        left, top = bounding_box[0], bounding_box[1]
        right, bottom = bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3]

        # coordinates of cropped image
        x1_crop = max(int(left), 0)
        y1_crop = max(int(top), 0)
        x2_crop = int(right)
        y2_crop = int(bottom)

        face = image[y1_crop:y2_crop, x1_crop:x2_crop, :]
        face = cv2.resize(face, (cf.IMAGE_SIZE, cf.IMAGE_SIZE), cv2.INTER_AREA)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face = face.reshape(cf.IMAGE_SIZE, cf.IMAGE_SIZE, 1)

        cropped_face[i, :, :, :] = face
        boxes.append((x1_crop, y1_crop, x2_crop, y2_crop))

    return cropped_face, boxes

In [ ]:
import numpy as np

# import data_utils
# import config as cf
# import utils


class Datasets(object):
    def __init__(self, trainable, test_data_type='public_test'):
        self.all_data = []
        self.trainable = trainable
        self.race_train = getRaceImage()
        self.age_train = getAgeImage()
        self.gender_train = getGenderImage()

        if not trainable:
            self.test_data_type = test_data_type

        self.convert_data_format()

    def gen(self):
        np.random.shuffle(self.all_data)
        batch_images = []
        batch_labels = []
        batch_indexes = []

        for i in range(len(self.all_data)):
            image, label, index = self.all_data[i]
            batch_images.append(image)
            batch_labels.append(label)
            batch_indexes.append(index)

            if len(batch_images) == BATCH_SIZE:
                yield batch_images, batch_labels, batch_indexes
                batch_images = []
                batch_labels = []
                batch_indexes = []

        if len(batch_images) > 0:
            yield batch_images, batch_labels, batch_indexes

    def convert_data_format(self):
        if self.trainable:
            # Race dataset
            for i in range(len(self.race_train) * 2):
                image = (self.race_train[i % 23165][0] - 128.0) / 255.0
                label = get_one_hot_vector(7, int(self.race_train[i % 23165][3]))
                index = 1.0
                self.all_data.append((image, label, index))
        
            # Age datasets
            for i in range(len(self.age_train)):
                image = (self.age_train[i][0] - 128.0) / 255.0
                label = get_one_hot_vector(7, int(self.age_train[i][1]))
                index = 3.0
                self.all_data.append((image, label, index))
           
            # Gender datasets
            for i in range(len(self.gender_train)):
                image = (self.gender_train[i][0] - 128.0) / 255.0
                label = get_one_hot_vector(7, int(self.gender_train[i][2]))
                index = 4.0
                self.all_data.append((image, label, index))

        else:
            # Smile datasets
            for i in range(len(self.smile_test)):
                image = (self.smile_test[i][0] - 128.0) / 255.0
                label = get_one_hot_vector(7, int(self.smile_test[i][3]))
                index = 1.0
                self.all_data.append((image, label, index))
       
            # Age datasets
            for i in range(len(self.age_test)):
                image = (self.age_test[i][0] - 128.0) / 255.0
                label = get_one_hot_vector(7, int(self.age_test[i][1]))
                index = 3.0
                self.all_data.append((image, label, index))
        
            # Gender datasets
            for i in range(len(self.gender_test)):
                image = (self.gender_test[i][0] - 128.0) / 255.0
                label = get_one_hot_vector(7, int(self.gender_test[i][2]))
                index = 4.0
                self.all_data.append((image, label, index))

In [ ]:
data=Datasets(trainable=True, test_data_type='public_test')


Loading Race image datasets.....
Done! 
Number of race train data:  23165

Loading age image datasets.....
Done!
Number of age train data:  45000

Loading gender image datasets.....
Done!
Number of gender train data:  45000


# Model(BKNet)

In [ ]:
tf.compat.v1.disable_eager_execution()


class Model(object):
    def __init__(self, session, trainable=True, prediction=False):
        self.global_step = tf.compat.v1.get_variable(name='global_step', initializer=tf.constant(0), trainable=False)
        self.batch_size = BATCH_SIZE
        self.sess = session
        self.model_dir = MODEL_DIR
        self.trainable = trainable
        self.prediction = prediction
        self.num_epochs = EPOCHS

        # Building model
        self._define_input()
        self._build_model()

        if not prediction:
            self._define_loss()
            # Learning rate and train op
            # learning_rate = tf.train.exponential_decay(learning_rate=config.INIT_LR, global_step=self.global_step,
            # decay_steps=config.DECAY_STEP, decay_rate=config.DECAY_LR, staircase=True)
            self.train_step = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3).minimize(self.total_loss,
                                                                                            global_step=self.global_step)
            # Input data
            self.data = Datasets(trainable=self.trainable, test_data_type='public_test')

        # Init checkpoints
        self.saver_all = tf.compat.v1.train.Saver(tf.compat.v1.global_variables(), max_to_keep=5)
        self.checkpoint_path = os.path.join(self.model_dir, 'model.ckpt')
        ckpt = tf.train.get_checkpoint_state(self.model_dir)

        if ckpt:
            print('Reading model parameters from %s', ckpt.model_checkpoint_path)
            self.saver_all.restore(self.sess, ckpt.model_checkpoint_path)
        else:
            print('Created model with fresh parameters.')
            self.sess.run(tf.compat.v1.global_variables_initializer())

    def _define_input(self):
        self.input_images = tf.compat.v1.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, 3])
        self.keep_prob = tf.compat.v1.placeholder(tf.float32)
        self.phase_train = tf.compat.v1.placeholder(tf.bool)
        if not self.prediction:
            self.input_labels = tf.compat.v1.placeholder(tf.float32, [None, 7])
            self.input_indexes = tf.compat.v1.placeholder(tf.float32, [None])

    def _build_model(self):
        # Extract features
        x = VGG_ConvBlock('Block1', self.input_images, 3, 32, 2, 1, self.phase_train)
        x = VGG_ConvBlock('Block2', x, 32, 64, 2, 1, self.phase_train)
        x = VGG_ConvBlock('Block3', x, 64, 128, 2, 1, self.phase_train)
        x = VGG_ConvBlock('Block4', x, 128, 256, 3, 1, self.phase_train)

        # Race branch
        race_fc1 = FC('race_fc1', x, 256, self.keep_prob)
        race_fc2 = FC('race_fc2', race_fc1, 256, self.keep_prob)
        self.y_race_conv = FC('race_softmax', race_fc2, 5, self.keep_prob, 'softmax')

        # Gender branch
        gender_fc1 = FC('gender_fc1', x, 256, self.keep_prob)
        gender_fc2 = FC('gender_fc2', gender_fc1, 256, self.keep_prob)
        self.y_gender_conv = FC('gender_softmax', gender_fc2, 2, self.keep_prob, 'softmax')

        # Age branch
        age_fc1 = FC('age_fc1', x, 256, self.keep_prob)
        age_fc2 = FC('age_fc2', age_fc1, 256, self.keep_prob)
        self.y_age_conv = FC('age_softmax', age_fc2, 3, self.keep_prob, 'softmax')

    def _define_loss(self):
        self.race_mask = tf.cast(tf.equal(self.input_indexes, 1.0), tf.float32)
        self.age_mask = tf.cast(tf.equal(self.input_indexes, 3.0), tf.float32)
        self.gender_mask = tf.cast(tf.equal(self.input_indexes, 4.0), tf.float32)

        self.y_race = self.input_labels[:, :5]
        self.y_age = self.input_labels[:, :3]
        self.y_gender = self.input_labels[:, :2]

        # Extra variables
        race_correct_prediction = tf.equal(tf.argmax(self.y_race_conv, 1), tf.argmax(self.y_race, 1))
        age_correct_prediction = tf.equal(tf.argmax(self.y_age_conv, 1), tf.argmax(self.y_age, 1))
        gender_correct_prediction = tf.equal(tf.argmax(self.y_gender_conv, 1), tf.argmax(self.y_gender, 1))

        self.race_true_pred = tf.reduce_sum(tf.cast(race_correct_prediction, dtype=tf.float32) * self.race_mask)
        self.age_true_pred = tf.reduce_sum(tf.cast(age_correct_prediction, dtype=tf.float32) * self.age_mask)
        self.gender_true_pred = tf.reduce_sum(tf.cast(gender_correct_prediction, dtype=tf.float32) * self.gender_mask)

        self.race_cross_entropy = tf.reduce_mean(
            tf.reduce_sum(-self.y_race * tf.math.log(tf.clip_by_value(tf.nn.softmax(self.y_race_conv), 1e-10, 1.0)),
                          axis=1) * self.race_mask)

        self.age_cross_entropy = tf.reduce_mean(
            tf.reduce_sum(-self.y_age * tf.math.log(tf.clip_by_value(tf.nn.softmax(self.y_age_conv), 1e-10, 1.0)),
                          axis=1) * self.age_mask)

        self.gender_cross_entropy = tf.reduce_mean(
            tf.reduce_sum(-self.y_gender * tf.math.log(tf.clip_by_value(tf.nn.softmax(self.y_gender_conv), 1e-10, 1.0)),
                          axis=1) * self.gender_mask)

        # Add l2 regularizer
        l2_loss = []
        for var in tf.compat.v1.trainable_variables():
            if var.op.name.find(r'DW') > 0:
                l2_loss.append(tf.nn.l2_loss(var))

        self.l2_loss = WEIGHT_DECAY * tf.add_n(l2_loss)

        self.total_loss = self.race_cross_entropy + self.gender_cross_entropy + self.l2_loss + self.age_cross_entropy

    @staticmethod
    def count_trainable_params():
        total_parameters = 0
        for variable in tf.compat.v1.trainable_variables():
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        print("Total training params: %.1fM" % (total_parameters / 1e6))

    def train(self):
        for epoch in range(self.num_epochs):
            race_nb_true_pred = 0
            age_nb_true_pred = 0
            gender_nb_true_pred = 0

            print("=======================================================================")
            print('Epoch %d/%d: ' % (epoch + 1, EPOCHS))
            for batch_image, batch_label, batch_index in self.data.gen():
                feed_dict = {self.input_images: batch_image,
                             self.input_labels: batch_label,
                             self.input_indexes: batch_index,
                             self.keep_prob: 0.5,
                             self.phase_train: self.trainable
                             }

                ttl, rcl, agl, gel, l2l, _ = self.sess.run([self.total_loss, self.race_cross_entropy,
                                                            self.age_cross_entropy,
                                                            self.gender_cross_entropy, self.l2_loss,
                                                            self.train_step], feed_dict=feed_dict)

                race_nb_true_pred += self.sess.run(self.race_true_pred, feed_dict=feed_dict)
                age_nb_true_pred += self.sess.run(self.age_true_pred, feed_dict=feed_dict)
                gender_nb_true_pred += self.sess.run(self.gender_true_pred, feed_dict=feed_dict)

                print('race_loss: %.2f, age_loss: %.2f, gender_loss: %.2f, l2_loss: %.2f, total_loss: %.2f\r' % (
                    rcl, agl, gel, l2l, ttl), end="")

            race_nb_train = len(self.data.race_train) * 10
            age_nb_train = len(self.data.age_train)
            gender_nb_train = len(self.data.gender_train)

            race_train_acc = race_nb_true_pred * 1.0 / race_nb_train
            age_train_acc = age_nb_true_pred * 1.0 / age_nb_train
            gender_train_acc = gender_nb_true_pred * 1.0 / gender_nb_train

            print('\n')
            print('Race task train accuracy: ', str(race_train_acc * 100))
            print('Age task train accuracy: ', str(age_train_acc * 100))
            print('Gender task train accuracy: ', str(gender_train_acc * 100))

            self.saver_all.save(self.sess, self.model_dir + '/model.ckpt')

    def test(self):
        # Evaluate model on the test data

        race_nb_true_pred = 0
        age_nb_true_pred = 0
        gender_nb_true_pred = 0

        for batch_image, batch_label, batch_index in self.data.gen():
            feed_dict = {self.input_images: batch_image,
                         self.input_labels: batch_label,
                         self.input_indexes: batch_index,
                         self.keep_prob: 1,
                         self.phase_train: self.trainable}

            race_nb_true_pred += self.sess.run(self.race_true_pred, feed_dict)
            age_nb_true_pred += self.sess.run(self.age_true_pred, feed_dict)
            gender_nb_true_pred += self.sess.run(self.gender_true_pred, feed_dict)

        race_nb_test = len(self.data.race_test)
        age_nb_test = len(self.data.age_test)
        gender_nb_test = len(self.data.gender_test)

        race_test_accuracy = race_nb_true_pred * 1.0 / race_nb_test
        gender_test_accuracy = gender_nb_true_pred * 1.0 / gender_nb_test
        age_test_accuracy = age_nb_true_pred * 1.0 / age_nb_test

        print('\nResult: ')
        print('Race task test accuracy: ' + str(race_test_accuracy * 100))
        print('Gender task test accuracy: ' + str(gender_test_accuracy * 100))
        print('Age task test accuracy: ' + str(age_test_accuracy * 100))

    def predict(self, image):
        # M is MALE and F is Female
        RACE_DICT = {0: 'White', 1: 'Black', 2:'Asian', 3:'Indian', 4:'Others'}
        GENDER_DICT = {0: 'M', 1: 'F'}
        AGE_DICT = {0: '1-26', 1: '27-52', 2: '53-80'}
        labels = []

        feed_dict = {self.input_images: image,
                     self.keep_prob: 0,
                     self.phase_train: self.trainable}

        race_prediction_idx = self.sess.run(tf.argmax(self.y_race_conv, axis=1), feed_dict=feed_dict)
        age_prediction_idx = self.sess.run(tf.argmax(self.y_age_conv, axis=1), feed_dict=feed_dict)
        gender_prediction_idx = self.sess.run(tf.argmax(self.y_gender_conv, axis=1), feed_dict=feed_dict)

        for i in range(len(race_prediction_idx)):
            race_label = RACE_DICT[race_prediction_idx[i]]
            age_label = AGE_DICT[age_prediction_idx[i]]
            gender_label = GENDER_DICT[gender_prediction_idx[i]]

            labels.append((race_label, age_label, gender_label))

        return labels

In [ ]:
session = tf.compat.v1.Session()

In [ ]:
multitask_model = Model(session, trainable=True)


Loading Race image datasets.....
Done! 
Number of race train data:  23165

Loading age image datasets.....
Done!
Number of age train data:  45000

Loading gender image datasets.....
Done!
Number of gender train data:  45000
Created model with fresh parameters.


In [ ]:
multitask_model.train()

Epoch 1/80: 
race_loss: 0.44, age_loss: 0.29, gender_loss: 0.21, l2_loss: 0.08, total_loss: 1.03

Race task train accuracy:  9.110727390459745
Age task train accuracy:  66.61555555555555
Gender task train accuracy:  70.15333333333334
Epoch 2/80: 
race_loss: 0.54, age_loss: 0.24, gender_loss: 0.25, l2_loss: 0.04, total_loss: 1.06

Race task train accuracy:  9.779408590546081
Age task train accuracy:  66.61555555555555
Gender task train accuracy:  73.34444444444445
Epoch 3/80: 
race_loss: 0.47, age_loss: 0.24, gender_loss: 0.14, l2_loss: 0.03, total_loss: 0.88

Race task train accuracy:  10.490826678178285
Age task train accuracy:  66.61555555555555
Gender task train accuracy:  77.33555555555556
Epoch 4/80: 
race_loss: 0.43, age_loss: 0.30, gender_loss: 0.16, l2_loss: 0.03, total_loss: 0.93

Race task train accuracy:  11.394344916900497
Age task train accuracy:  66.61555555555555
Gender task train accuracy:  81.89111111111112
Epoch 5/80: 
race_loss: 0.45, age_loss: 0.30, gender_loss: 0.1